In [1]:
#Import standard packages
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import itertools

from scipy import io as sio
from os.path import dirname, join as pjoin
from scipy import stats
from scipy.io import savemat
from sklearn.calibration import CalibrationDisplay

from sklearn import linear_model
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier


/opt/anaconda3/envs/Neural-Decoding/lib/python3.11/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [3]:
def get_test_train_splits(data, decisions, n_folds,n_reps,rng_state_test): 
    
    skf = RepeatedStratifiedKFold(n_splits=n_folds,n_repeats=n_reps,random_state=rng_state_test)
    x_temp=np.zeros([len(decisions),2]);
    
    training_sets=[data[ind_train,:] for (ind_train, ind_test) in skf.split(x_temp, decisions)]
    training_Y=[decisions[ind_train] for (ind_train, ind_test) in skf.split(x_temp, decisions)]
    
    val_sets=[data[ind_test,:] for (ind_train, ind_test) in skf.split(x_temp, decisions)]
    val_Y=[decisions[ind_test] for (ind_train, ind_test) in skf.split(x_temp, decisions)]

    return (training_sets, training_Y), (val_sets, val_Y)

In [4]:
n_test_train_fold=5
n_test_train_rep=2
n_val_train_fold=4
Cs_to_test = np.logspace(3,-6,10)

# Define the classifiers to be compared in the study.
lr = LogisticRegressionCV(Cs=Cs_to_test, cv=n_val_train_fold, scoring="accuracy", max_iter=1_000,n_jobs=-1) 
lda = LinearDiscriminantAnalysis(solver = 'lsqr',shrinkage = 'auto')
gnb = GaussianNB()
svc = GridSearchCV(estimator=SVC(), param_grid={'C': Cs_to_test}, scoring='accuracy', cv=n_val_train_fold, n_jobs=-1)
rfc = RandomForestClassifier(random_state=42)

clf_list = [
    (lr, "Logistic Regression"),
    (lda, "LDA"),    
    (gnb, "Naive Bayes"),
    (svc, "SVC"),
    (rfc, "Random forest"),
]
n_clf=5

In [7]:
# load matlab file for spikes (Nneuron*Ncondition_Ntrials)
sessionidx=['session6_','session8_','session10_','session13_'];
data_dir = '/Users/shushu/Dropbox/project/ori_decoding/data/'
idx_session=2;
mat_fname = pjoin(data_dir,sessionidx[idx_session]+'spikes_avg_norm.mat');
mat_contents = sio.loadmat(mat_fname)
spikes=mat_contents['spikes_avg_norm']

mat_fname2 = pjoin(data_dir,sessionidx[idx_session]+'prefidx_offset.mat');
mat_contents2 = sio.loadmat(mat_fname2)
prefidx_offset0=mat_contents2['prefidx_offset']
prefidx_offset0=prefidx_offset0[0]

n_neurons=spikes.shape[0]
ori_allpair = np.array(list(itertools.combinations(range(18), 2)))
n_pair=ori_allpair.shape[0]
n_trials=2*spikes.shape[2]; #double the trail number if we combine the two directions for each orientation
decisions=np.reshape([np.zeros(n_trials),np.ones(n_trials)],(-1))
print(spikes.shape)

(131, 144, 5)


In [ ]:
test_accuracy=np.zeros((n_clf,n_test_train_fold*n_test_train_rep,n_neurons,n_pair))+0.5

for id_neuron in range(n_neurons):
    id0=max(id_neuron-5,0);
    if id0+10>n_neurons:
        id0=n_neurons-10;
    c_prefidx=prefidx_offset0[id_neuron]
    c_spike_pref=np.concatenate((spikes[id0:id0+10,c_prefidx:c_prefidx+18,:],spikes[id0:id0+10,c_prefidx+18:c_prefidx+36,:]),axis=2)
    for id_pair in range(n_pair):
        c_spike=np.concatenate((c_spike_pref[:,ori_allpair[id_pair][0],:],c_spike_pref[:,ori_allpair[id_pair][1],:]),axis=1).T
        # now c_spike (n oritentation trials *10 neurons) is the spike file, for selected decoding ori pair, and selected groups
        (training_sets, training_Ys), (test_sets, test_Ys) = get_test_train_splits(c_spike, decisions,n_test_train_fold,n_test_train_rep,rng_state_test=202406144)
        test_scores = []

        # Iterate through the k=5 folds
        for fold in range(n_test_train_fold*n_test_train_rep):
          
            test_X = test_sets[fold]
            test_Y = test_Ys[fold]          
            training_X = training_sets[fold]
            training_Y = training_Ys[fold]                    
            
            # randoem shuffle the spikes along trials to disrupt the noise correlations
            # training_X0=training_X[training_Y==0,:];
            # training_X1=training_X[training_Y==1,:];
            # for id_x in range(10):
            #     np.random.shuffle(training_X0[:,id_x])
            #     np.random.shuffle(training_X1[:,id_x])
            # training_X=np.concatenate((training_X0,training_X1),axis=0)
            # training_Y=np.sort(training_Y)
            
            for idx_clf, (clf, name) in enumerate(clf_list):
                this_whole_model=clf.fit(training_X, training_Y)           
                this_clf_accuracy = this_whole_model.score(test_X, test_Y)
                test_accuracy[idx_clf,fold,id_neuron,id_pair]= this_clf_accuracy
    print(id_neuron,np.mean(test_accuracy[:,:,id_neuron,:],axis=(1,2)))
mean_test_accuracy=np.mean(test_accuracy,axis=1)             


/opt/anaconda3/envs/Neural-Decoding/lib/python3.11/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
/opt/anaconda3/envs/Neural-Decoding/lib/python3.11/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https:/

In [ ]:
savemat_fname = pjoin(data_dir,sessionidx[idx_session]+'all_raw_accuracy.mat');
savemat(savemat_fname,{'test_accuracy':test_accuracy})
savemat_fname = pjoin(data_dir,sessionidx[idx_session]+'mean_raw_accuracy.mat');
savemat(savemat_fname,{'mean_test_accuracy':mean_test_accuracy})


# savemat_fname = pjoin(data_dir,sessionidx[idx_session]+'all_shuffle_accuracy.mat');
# savemat(savemat_fname,{'test_accuracy':test_accuracy})
# savemat_fname = pjoin(data_dir,sessionidx[idx_session]+'mean_shuffle_accuracy.mat');
# savemat(savemat_fname,{'mean_test_accuracy':mean_test_accuracy})

In [ ]:
mean_test_accuracy_byneuron=np.mean(mean_test_accuracy,axis=2)

colors = plt.get_cmap("Dark2")

fig, axs=plt.subplots(figsize=(10,5))
for idx_clf, (clf, name)  in enumerate(clf_list):
    plt.plot(np.arange(0,n_neurons),mean_test_accuracy_byneuron[idx_clf,:])
axs.legend(['lr','LDA','NGB','SVM','RF'])
plt.xlabel("Neuron (ID)")
plt.ylabel("Mean accuracy")
#plt.ylim([0.5,1])
